In [ ]:
import pandas as pd
from scipy import stats

In [ ]:
import pandas as pd
from scipy import stats
import json

#Load protein dictionaries
with open('shortened_protein_dict.json') as file:
    protein_dict = json.load(file)

with open('shortened_protein_translations.json') as file:
    protein_translations = json.load(file)
    
with open('protein_translations.json') as file:
    full_protein_translations = json.load(file)
    
with open('protein_dict.json') as file:
    full_protein_dict = json.load(file)

In [ ]:
phages_df = pd.read_csv('CombinedData.csv')
#sequenced_df = pd.read_csv('SequencedPhages.csv')
#sequenced_df = sequenced_df[['Phage Name', 'Host']]

#phages_df = pd.merge(phages_df, sequenced_df, on='Phage Name')

In [ ]:
host_list = phages_df['Host'].value_counts()
host_list = host_list[host_list > 30]
print(host_list)

In [ ]:
#host_results = pd.DataFrame({'Host 1': [], 'Host 2': [], 'Factor': [], 
                             'P-value': [], 'Difference': []})

In [ ]:
factors = ['AQI', 'GHI', 'pH', 'Population Density', 'Dew Point', 'Albedo', 'GC%', 'Genome Length(bp)']

#Iterate through host pairs
for host1 in host_list.index:
    host1_df = phages_df[phages_df['Host'] == host1]
    for host2 in host_list.index:
        host2_df = phages_df[phages_df['Host'] == host2]
        #Make sure pairs of hosts are only counted once
        if host_list[host1] > host_list[host2]:
            #Iterate through factors
            for factor in factors:
                p_value = stats.ttest_ind(host1_df[factor], host2_df[factor], nan_policy='omit')[1]
                #If significant difference in means, record results
                if p_value < 0.001:
                    diff = host1_df[factor].mean() - host2_df[factor].mean()
                    
                    temp_df = pd.DataFrame({'Host 1': [host1], 'Host 2': [host2], 'Factor': [factor], 
                             'P-value': [p_value], 'Difference': [diff]})
                    
                    host_results = host_results.append(temp_df, ignore_index='False')
                    
                    
                    
                    
                    print(factor)
                    print(f"{host1}, {host2}")
                    print(f"P-value: {p_value}")
                    print(f"Difference: {diff}")
                    print()

In [ ]:
host_results.to_csv('Host Correlations.csv', index=False)

In [ ]:
gene_host_results = host_results[(host_results['Factor'] == 'Genome Length(bp)') | (host_results['Factor'] == 'GC%')]

In [ ]:
gene_host_results.to_csv('Host Genome Associations.csv', index=False)